'''.  
    Use 2 ways: 1. hit rate 2. minisearch.   
'''. 

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import json
from minsearch import Index
import requests

In [ ]:
df_evaluate = pd.read_csv('prepared_question_evaluation.csv')
df_evaluate = df_evaluate.head(4000)
grouped_truth = df_evaluate.to_dict(orient='records')

In [ ]:
with open("document.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

In [ ]:
# Create and fit the index
index = Index(
    text_fields=["question", "text", "section", "id"],
    keyword_fields=["course"]
)
index.fit(documents)

In [ ]:
def get_rs_from_minsearch(query, course, id):
    filter_dict = {"course": course}
    boost_dict = {"question": 3, "section": 0.5}
    results = index.search(query, 
                        filter_dict=filter_dict, 
                        boost_dict=boost_dict, 
                        num_results = 5)
    rs = []
    for each in results:
        rs.append(each['id'] == id)
    return rs

In [ ]:
relevance_total = []
for q in tqdm(grouped_truth):
    doc_id = q['doc_id']
    results = get_rs_from_minsearch(q['question'], q['course'], doc_id)
    relevance_total.append(results)

In [ ]:
def hit_rate(relevance_total):
    rate = 0
    for each in relevance_total:
        if True in each:
            rate +=1
    return rate/len(relevance_total)

In [ ]:
def mrr(relevance_total):
    score = 0
    for each in relevance_total:
        for i in range(len(each)):
            if each[i] == True:
                score += 1/(i+1)
    return score/len(relevance_total)

## elastic search

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
def elastic_search(query,course):
    pass

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)